In [12]:
import json
from lxml import objectify
from selenium import webdriver
from bs4 import BeautifulSoup

import urllib
from urllib.request import urlopen
from urllib.error import HTTPError
from datetime import datetime
from datetime import timedelta
import time
import pandas
import re

## Links and params

In [2]:
## LTA
headers = { 'AccountKey' : 'LBdfS4+RSEi4witEa6RjjQ==',
            'UniqueUserID' : '812eb03f-f3a1-421c-96de-069c59844bbb',
            'accept' : 'application/json'}
uri = 'http://datamall2.mytransport.sg/'
path_TSB = 'ltaodataservice/TrafficSpeedBands'
path_TI = 'ltaodataservice/TrafficIncidents'
path_FTL = 'ltaodataservice/FaultyTrafficLights'
path_ETT = 'ltaodataservice/EstTravelTimes'
path_RO = 'ltaodataservice/RoadOpenings'
path_RW = 'ltaodataservice/RoadWorks'
## NEA links and auth params
key = '781CF461BB6606ADC767F3B357E848ED4A6709A0B39C63D6'
urlNowcast = 'http://api.nea.gov.sg/api/WebAPI/?dataset=2hr_nowcast&keyref=' + key
urlWarning = 'http://api.nea.gov.sg/api/WebAPI/?dataset=heavy_rain_warning&keyref=' + key
## Meteorological Service Singapore Rainfall observations
urlRainfall = 'http://www.weather.gov.sg/weather-currentobservations-rainfall'

### Test LTA data EPN

In [6]:
# Traffic Speed Band data - updates every 5 minutes
request = urllib.request.Request(uri + path_TSB, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTSB = pandas.DataFrame(messages)

In [17]:
dfTSB.head(2)

,LinkID,Location,MaximumSpeed,MinimumSpeed,RoadCategory,RoadName,SpeedBand
0,103098943,1.3805895432850994 103.74201730473243 1.380040...,39,20,D,CHOA CHU KANG AVENUE 3,2
1,103076063,1.399027473240379 103.7553182996927 1.39866097...,39,20,B,WOODLANDS ROAD,2


In [9]:
# ESTIMATED TRAVEL TIMES data - updates every 5 minutes
request = urllib.request.Request(uri + path_ETT, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfETT = pandas.DataFrame(messages)

In [18]:
dfETT.head(2)

,Direction,EndPoint,EstTime,FarEndPoint,Name,StartPoint
0,1,TELOK BLANGAH RD,2,TUAS CHECKPOINT,AYE,AYE/MCE INTERCHANGE
1,1,LOWER DELTA RD,1,TUAS CHECKPOINT,AYE,TELOK BLANGAH RD


In [13]:
# collect Traffic Incidents data - updates every 2 minutes
request = urllib.request.Request(uri + path_TI, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTI = pandas.DataFrame(messages)

In [19]:
dfTI.head(2)

,Latitude,Longitude,Message,Type
0,1.392957,103.824566,(8/11)22:09 Vehicle breakdown on SLE (towards ...,Vehicle breakdown
1,1.305273,103.860494,(8/11)22:05 Roadworks on Victoria Street (towa...,Roadwork


In [72]:
# collect Nowcast data - updates every 30 mins
request = urllib.request.Request(urlNowcast)
try:
    parsed = objectify.parse(urlopen(request))
except HTTPError as e:
    print (e)
else:
    parsed = objectify.parse(urlopen(request))
    root = parsed.getroot()
    dfNowcast = pandas.DataFrame(columns=['Forecast', 'Latitude', 'Longitude', 'Name'])
    for area in root.iter('area'):
        dfNowcast = dfNowcast.append({'Forecast':area.get('forecast'),'Latitude':area.get('lat'),'Longtitude':area.get('lon'),'Name':area.get('name')}, ignore_index=True)

In [73]:
dfNowcast.head(2)

,Forecast,Latitude,Longitude,Name,Longtitude
0,FN,1.37500000,NaN,Ang Mo Kio,103.83900000
1,FN,1.32100000,NaN,Bedok,103.92400000


In [80]:
# collect Rainfall data - updates every 10 mins
driver = webdriver.PhantomJS(executable_path='../bin/phantomjs')
try:
        timeNow = datetime.now().get(urlRainfall)
except HTTPError as e:
    print (e)
else:
    # wait for page load
    time.sleep(2)
    html30mins = driver.page_source
    bsObj = BeautifulSoup(html30mins,"lxml")
    # Get data timestamp
    element = bsObj.find("img", {"id":"basemap"})
    pattern = re.compile('[0-9]+:[0-9]+')
    rainfallTimestamp = pattern.findall(element.attrs['src'])[0]
    # load 30 mins rainfall data
    dataset30mins =  bsObj.findAll("",{"class":"sgr"})
    dfRainfall = pandas.DataFrame(columns=['StationId', 'rain30mins'])
    for data in dataset30mins:
        dfRainfall = dfRainfall.append({'StationId':data.get('id'), 'rain30mins':data.get_text()},ignore_index=True)

In [81]:
dfRainfall.head(2)

,StationId,rain30mins
0,S07,0
1,S101,0
